# extract Legacy Survey photometry for BGS targets

In [2]:
import os
import numpy as np 
import astropy.table as atable
from tqdm.notebook import tqdm, trange

In [3]:
from desitarget import cuts as desi_cuts

In [4]:
# -- plotting -- 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

In [5]:
import corner as DFM

In [6]:
ls_S_dir = '/global/cfs/cdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0'
ls_N_dir = '/global/cfs/cdirs/cosmo/data/legacysurvey/dr9/north/sweep/9.0'

cat_S = atable.Table.read(os.path.join(ls_S_dir, 'sweep-200p005-210p010.fits'))
cat_N = atable.Table.read(os.path.join(ls_N_dir, 'sweep-190p040-200p045.fits'))

In [7]:
cat_S['PHOTSYS'] = 'S'
cat_N['PHOTSYS'] = 'N'

In [8]:
def BGS_quality_mask(targets, south=True):
    ''' apply BGS quality cut and imaging mask
    '''
    photsys_north, photsys_south, obs_rflux, gflux, rflux, zflux,                 \
    w1flux, w2flux, gfiberflux, rfiberflux, zfiberflux,                           \
    gfibertotflux, rfibertotflux, zfibertotflux,                                  \
    objtype, release, ra, dec, gfluxivar, rfluxivar, zfluxivar, w1fluxivar,       \
    gnobs, rnobs, znobs, gfracflux, rfracflux, zfracflux,                         \
    gfracmasked, rfracmasked, zfracmasked,                                        \
    gfracin, rfracin, zfracin, gallmask, rallmask, zallmask,                      \
    gsnr, rsnr, zsnr, w1snr, w2snr, dchisq, deltaChi2, maskbits, refcat = desi_cuts._prepare_optical_wise(objects=targets, mask=True)

    gaia, pmra, pmdec, parallax, parallaxovererror, parallaxerr, gaiagmag, \
    gaiabmag, gaiarmag, gaiaaen, gaiadupsource, Grr, gaiaparamssolved, \
    gaiabprpfactor, gaiasigma5dmax, galb = desi_cuts._prepare_gaia(objects=targets, colnames=None)

    bgs = np.ones_like(gnobs, dtype='?')    
    bgs &= (gnobs >= 1) & (rnobs >= 1) & (znobs >= 1)
    bgs &= (gfluxivar > 0) & (rfluxivar > 0) & (zfluxivar > 0)
    
    # ADM geometric masking cuts from the Legacy Surveys.
    # BRIGHT & CLUSTER for BGS
    bgs &= desi_cuts.imaging_mask(maskbits, bgsmask=True)

    return bgs

In [9]:
qm_S = BGS_quality_mask(cat_S, south=True)
qm_N = BGS_quality_mask(cat_N, south=True)

In [10]:
optw_S = desi_cuts._prepare_optical_wise(objects=cat_S, mask=True)
optw_N = desi_cuts._prepare_optical_wise(objects=cat_N, mask=True)

gaia_S = desi_cuts._prepare_gaia(objects=cat_S, colnames=None)
gaia_N = desi_cuts._prepare_gaia(objects=cat_N, colnames=None)

In [11]:
rfib_S = 22.5 - 2.5 * np.log10(optw_S[9].clip(1e-16))
gmag_S = 22.5 - 2.5 * np.log10(optw_S[3].clip(1e-16))
rmag_S = 22.5 - 2.5 * np.log10(optw_S[4].clip(1e-16))
zmag_S = 22.5 - 2.5 * np.log10(optw_S[5].clip(1e-16))
w1_S = 22.5 - 2.5 * np.log10(optw_S[6].clip(1e-16))

gaia_g_S = gaia_S[6]
G_rr_S = gaia_S[11]

rfib_N = 22.5 - 2.5 * np.log10(optw_N[9].clip(1e-16))
gmag_N = 22.5 - 2.5 * np.log10(optw_N[3].clip(1e-16))
rmag_N = 22.5 - 2.5 * np.log10(optw_N[4].clip(1e-16))
zmag_N = 22.5 - 2.5 * np.log10(optw_N[5].clip(1e-16))
w1_N = 22.5 - 2.5 * np.log10(optw_N[6].clip(1e-16))

gaia_g_N = gaia_N[6]
G_rr_N = gaia_N[11]

In [12]:
def BGS_sample(targets, south=True, targtype='bright'):
    ''' apply BGS quality cut and imaging mask
    '''
    photsys_north, photsys_south, obs_rflux, gflux, rflux, zflux,                 \
    w1flux, w2flux, gfiberflux, rfiberflux, zfiberflux,                           \
    gfibertotflux, rfibertotflux, zfibertotflux,                                  \
    objtype, release, ra, dec, gfluxivar, rfluxivar, zfluxivar, w1fluxivar,       \
    gnobs, rnobs, znobs, gfracflux, rfracflux, zfracflux,                         \
    gfracmasked, rfracmasked, zfracmasked,                                        \
    gfracin, rfracin, zfracin, gallmask, rallmask, zallmask,                      \
    gsnr, rsnr, zsnr, w1snr, w2snr, dchisq, deltaChi2, maskbits, refcat = desi_cuts._prepare_optical_wise(objects=targets, mask=True)

    gaia, pmra, pmdec, parallax, parallaxovererror, parallaxerr, gaiagmag, \
    gaiabmag, gaiarmag, gaiaaen, gaiadupsource, Grr, gaiaparamssolved, \
    gaiabprpfactor, gaiasigma5dmax, galb = desi_cuts._prepare_gaia(objects=targets, colnames=None)
    
    bgs = desi_cuts.isBGS(
            gflux=gflux, rflux=rflux, zflux=zflux, w1flux=w1flux, w2flux=w2flux, rfibertotflux=rfibertotflux,
            rfiberflux=rfiberflux, gnobs=gnobs, rnobs=rnobs, znobs=znobs,
            gfluxivar=gfluxivar, rfluxivar=rfluxivar, zfluxivar=zfluxivar,
            maskbits=maskbits, Grr=Grr, refcat=np.array(refcat).astype(str), w1snr=w1snr, w2snr=w2snr, gaiagmag=gaiagmag,
            objtype=objtype, primary=None, south=south, targtype=targtype)
    return bgs

In [13]:
bgs_bright_S = BGS_sample(cat_S, south=True, targtype='bright')
bgs_bright_N = BGS_sample(cat_N, south=False, targtype='bright')

In [14]:
bgs_faint_S = BGS_sample(cat_S, south=True, targtype='faint')
bgs_faint_N = BGS_sample(cat_N, south=False, targtype='faint')

In [22]:
np.save('/global/cscratch1/sd/chahah/gqp_mc/legacysurvey/bgs.gmag.npy', np.concatenate([gmag_S[bgs_bright_S | bgs_faint_S], gmag_N[bgs_bright_N | bgs_faint_N]]))
np.save('/global/cscratch1/sd/chahah/gqp_mc/legacysurvey/bgs.rmag.npy', np.concatenate([rmag_S[bgs_bright_S | bgs_faint_S], rmag_N[bgs_bright_N | bgs_faint_N]]))
np.save('/global/cscratch1/sd/chahah/gqp_mc/legacysurvey/bgs.zmag.npy', np.concatenate([zmag_S[bgs_bright_S | bgs_faint_S], zmag_N[bgs_bright_N | bgs_faint_N]]))